In [1]:
import pandas as pd
import os
from dotenv import load_dotenv
import os
from difflib import get_close_matches
from functools import reduce
from difflib import SequenceMatcher

In [2]:
# Load environment variables from .env file
load_dotenv()

# Get API key from environment variables
API_KEY = os.getenv('API_KEY')

Reference: https://support.qs.com/hc/en-gb/articles/4410488025106-QS-World-University-Rankings-by-Subject
### The rankings columns are:
* Academic Reputation (30% weight)
-- The Academic Reputation (AR) indicator measures the reputation of institutions and their programmes by asking academic experts to nominate universities based on their subject area of expertise. Pioneered by QS in 2004, it asks the question: which universities are demonstrating academic excellence? To answer this we collect and distil the collective intelligence of academics from around the world via our Academic Survey, evaluating nominations for approximately 7000 institutions each year.The indicator not only illuminates the quality of an institution's research, but also their approach to academic partnerships, their strategic impact, their educational innovativeness and the impact they have made on education and society at large.
The indicator is the centrepiece of almost all of the rankings across the QS portfolio. 

* Employer Reputation (15% weight)
-- The Employer Reputation (ER) indicator measures the reputation of institutions and their programmes among employers. We remain the only major ranking to focus on this vital aspect of a student's educational journey.

* Citations per Paper
-- The Citations per Paper (CPP) indicator measures the impact and quality of the scientific work done by institutions, on average per publication.

* H-Index
-- The h-index is an index that attempts to measure both the productivity and impact of the published work of a scientist or scholar. The index is based on the set of the scientist’s most cited papers and the number of citations that they have received in other publications. It can also be applied to the productivity and impact of a group of scientists, such as a department, or an institution (as in the case of our indicator), or a country, as well as a scholarly journal. The index is defined as the maximum value of h such that the given entity (author, journal, department, institution, etc.) has published at least h papers that have each been cited at least h times (https://doi.org/10.1073/pnas.0507655102). We use institution-level H Index.

* International Research Network
-- International Research Network (IRN) is a measure of an institution's success in creating and sustaining research partnerships with institutions in other locations. The indicator measures how diverse and rich an institution's research network is by looking at the number of different countries represented, and whether these relationships are renewed and repeated. We only consider sustained partnerships, defined as those which result in three or more joint papers published in a five-year period.


In [3]:
def get_top_us_institutions(file_path='../top_us_institutions.csv'):
    df_top = pd.read_csv(file_path)

    # print the first 5 rows
    print(df_top.head())

    # print the shape of the dataframe
    print(df_top.shape)

    # print the columns of the dataframe
    print(df_top.columns)

    school_names_list = df_top['school.name'].str.strip().values.tolist()
    print(school_names_list)

    return df_top, school_names_list

In [4]:
def get_unitids(file_path='hd2023.csv'):
    # -------------------------------
    #  Load IPEDS HD2023.csv
    # -------------------------------
    # Download from: https://nces.ed.gov/ipeds/datacenter/DataFiles.aspx → Institutional Characteristics → Header (HD2023.csv)

    if not os.path.exists(file_path):
        print("Download the file from https://nces.ed.gov/ipeds/datacenter/DataFiles.aspx → Institutional Characteristics → Header (HD2023.csv)")
        print("and save it to the raw_data folder")
        return pd.DataFrame()

    df_hd = pd.read_csv(file_path, low_memory=False, encoding='latin1')

    df_hd["INSTNM_clean"] = df_hd["INSTNM"].str.strip().str.lower()

    #assert('arizona state university' in df_hd["INSTNM_clean"])

    print(f"Loaded HD2023.csv with {len(df_hd)} institutions")

    return df_hd

In [5]:
# -------------------------------
# Institution Matching
# -------------------------------
def find_best_match(inst_clean: str, df_hd: pd.DataFrame) -> tuple[str, float]:
    """Find best match using prefix matching first, then fuzzy matching."""
    # Try prefix matching first
    prefix_matches = df_hd[df_hd["INSTNM_clean"].str.startswith(inst_clean)]
    if not prefix_matches.empty:
        return prefix_matches.iloc[0]["INSTNM_clean"], 1.0
    
    # If no prefix match, try fuzzy matching
    matches = get_close_matches(inst_clean, df_hd["INSTNM_clean"].tolist(), n=3, cutoff=0.6)
    if not matches:
        return "NO MATCH FOUND", 0.0
    
    # Get the best match based on word overlap
    best_match = None
    best_score = 0.0
    
    for match in matches:
        inst_words = set(inst_clean.split())
        match_words = set(match.split())
        common_words = inst_words.intersection(match_words)
        
        score = len(common_words) / max(len(inst_words), len(match_words))
        if score > best_score:
            best_score = score
            best_match = match
    
    return best_match, best_score

In [6]:
def match_institutions_unitid(my_institution_list, df_hd):
    matched_rows = []
    
    for inst in my_institution_list:
        inst_clean = inst.strip().lower()
        best_match, score = find_best_match(inst_clean, df_hd)
        
        if best_match != "NO MATCH FOUND" and score >= 0.5:
            unitid = df_hd[df_hd["INSTNM_clean"] == best_match]["UNITID"].values[0]
            matched_rows.append({
                "Institution": inst,
                "Matched_Name": best_match,
                "UNITID": unitid,
                "Match_Score": score
            })
        else:
            matched_rows.append({
                "Institution": inst,
                "Matched_Name": "NO MATCH FOUND",
                "UNITID": None,
                "Match_Score": 0.0
            })

   
    df_matches = pd.DataFrame(matched_rows)
    # make the unitid an integer before saving, output the list for subsequent use
    df_matches["UNITID"] = df_matches["UNITID"].fillna(-1).astype(int)
    # remove any rows where the unitid is -1
    df_matches = df_matches[df_matches["UNITID"] != -1]

    # output the list for subsequent use
    unitid_list = df_matches["UNITID"].tolist()

    # Display summary
    print("Matching complete!")
    print(f"Total institutions: {len(df_matches)}")
    print(f"Matches found: {df_matches['UNITID'].notnull().sum()}")

    # Show first 10 matches as preview
    df_matches.head(10)

    return df_matches, unitid_list

In [7]:
# Your list of institutions
df_top, my_institution_list = get_top_us_institutions()
print(f"Loaded {len(my_institution_list)} institutions")

print("Getting unitids for the institutions")
df_hd = get_unitids()

df_matches, unitid_list = match_institutions_unitid(my_institution_list, df_hd)
print(unitid_list)
# Save result
df_matches[["Institution", "UNITID"]].to_csv("institution_unitid_matched.csv", index=False)

   latest.admissions.admission_rate.overall  \
0                                    0.0345   
1                                    0.0756   
2                                    0.0391   
3                                    0.0474   
4                                    0.0450   

   latest.student.demographics.race_ethnicity.asian  \
0                                            0.2245   
1                                            0.2663   
2                                            0.2747   
3                                            0.3461   
4                                            0.2283   

   latest.student.demographics.race_ethnicity.white  \
0                                            0.3278   
1                                            0.2001   
2                                            0.2416   
3                                            0.2102   
4                                            0.3239   

   latest.student.demographics.race_ethnicity.hispanic 

#### At this point got to IPEDS website, harness the required data into various CSV files and come back here

`https://nces.ed.gov/ipeds/datacenter/InstitutionByName.aspx?stepId=1&sid=6d86013f-9157-4ca8-b55f-db58d3826513&rtid=5`

Proivde the unitid for list of relevant institutions, pick the following data columns manually (uugh!) and put it in `ipeds-all` directory as CSV files

In [15]:
def merge_ipeds_data(ipeds_dir="ipeds-all"):
    # Get all CSV files from ipeds-all directory
    csv_files = [f for f in os.listdir(ipeds_dir) if f.endswith('.csv')]

    # Read and merge all CSV files
    dfs = []
    for file in csv_files:
        file_path = os.path.join(ipeds_dir, file)
        df = pd.read_csv(file_path)
        
        # Standardize column names to lowercase
    # Read and merge all CSV files
    dfs = []
    for file in csv_files:
        file_path = os.path.join(ipeds_dir, file)
        df = pd.read_csv(file_path)
        
        # Standardize column names to lowercase
        df.columns = df.columns.str.lower()
        
        # Drop duplicate columns before adding to dfs list
        df = df.loc[:, ~df.columns.duplicated()]
        dfs.append(df)

    # Merge all dataframes on UNITID using reduce with suffixes to handle duplicate columns
    merged_df = reduce(
        lambda left, right: pd.merge(
            left, 
            right, 
            on='unitid', 
            how='outer',
            suffixes=('', f'_{len(dfs)}')  # Use empty suffix for left, numbered suffix for right
        ), 
        dfs
    )

    print(f"Successfully merged {len(csv_files)} files into iped_data.csv")
    print(f"Final shape: {merged_df.shape}")
    print(f"Columns: {merged_df.columns}")

    # drop any rows where the unitid is -1
    merged_df = merged_df[merged_df['unitid'] != -1]

    # rename relevant columns
    rename_dict = {
       'institution name': 'institution', 
       'sfa2223.percent of full-time first-time undergraduates awarded pell grants': 'pct_pell',
       'drvadm2023.percent admitted - total': 'pct_admitted',
       'drvadm2023.admissions yield - total': 'admissions_yield', 
       'drvic2023.tuition and fees, 2023-24': 'tuition_fees',
       'drvef2023.full-time undergraduate enrollment': 'enrollment_ft',
       'drvef2023.percent of total enrollment that are asian': 'pct_asian',
       'drvef2023.percent of total enrollment that are black or african american': 'pct_black',
       'drvef2023.percent of total enrollment that are hispanic/latino': 'pct_hispanic',
       'drvef2023.percent of total enrollment that are native hawaiian or other pacific islander': 'pct_native_hawaiian',
       'drvef2023.percent of total enrollment that are white': 'pct_white',
       'drvef2023.number of first-time undergraduates - in-state': 'enrollment_ft_in_state',
       'drvef2023.percent of first-time undergraduates - in-state': 'pct_enrollment_ft_in_state',
       'drvef2023.number of first-time undergraduates - out-of-state': 'enrollment_ft_out_of_state',
       'drvef2023.percent of first-time undergraduates - out-of-state': 'pct_enrollment_ft_out_of_state',
       'drvef2023.number of first-time undergraduates - foreign countries': 'enrollment_ft_foreign_countries',
       'drvef2023.percent of first-time undergraduates - foreign countries': 'pct_enrollment_ft_foreign_countries'
    }
    merge_df_relevant = merged_df.rename(columns=rename_dict)
    
    merge_df_relevant = merge_df_relevant[['unitid', 
                                           'institution', 
                                           'year', 
                                           'pct_pell', 
                                           'pct_admitted', 
                                           'admissions_yield', 
                                           'tuition_fees', 
                                           'enrollment_ft', 
                                           'pct_asian', 
                                           'pct_black', 
                                           'pct_hispanic', 
                                           'pct_native_hawaiian', 
                                           'pct_white', 
                                           'enrollment_ft_in_state', 
                                           'pct_enrollment_ft_in_state', 
                                           'enrollment_ft_out_of_state', 
                                           'pct_enrollment_ft_out_of_state', 
                                           'enrollment_ft_foreign_countries', 
                                           'pct_enrollment_ft_foreign_countries']]

    # Save merged dataframe
    merge_df_relevant.to_csv('iped_data.csv', index=False)

    return merge_df_relevant

#### Obtain salary details from CDS database

In [16]:
def get_salary_data():
    # read mrc_salary_data.csv
    salary_df = pd.read_csv("mrc_salary_table_cleaned.csv")

   # print the first 5 rows
    print(salary_df.head())

    # print the shape of the dataframe
    print(salary_df.shape)

    # print the columns of the dataframe
    print(salary_df.columns)

    return salary_df

In [17]:
ipeds_data = merge_ipeds_data(ipeds_dir="ipeds_all")

salary_data = get_salary_data()

Successfully merged 5 files into iped_data.csv
Final shape: (40, 45)
Columns: Index(['unitid', 'institution name', 'year',
       'sfa2223.percent of full-time first-time undergraduates awarded pell grants',
       'xpgrnt_p', 'institution name_5', 'year_5',
       'drvadm2023.percent admitted - total',
       'drvadm2023.admissions yield - total', 'institution name_5', 'year_5',
       'drvadm2023.percent admitted - total_5',
       'drvadm2023.admissions yield - total_5',
       'drvic2023.tuition and fees, 2023-24',
       'drvef2023.full-time undergraduate enrollment',
       'drvef2023.percent of total enrollment that are asian',
       'drvef2023.percent of total enrollment that are black or african american',
       'drvef2023.percent of total enrollment that are hispanic/latino',
       'drvef2023.percent of total enrollment that are native hawaiian or other pacific islander',
       'drvef2023.percent of total enrollment that are white',
       'drvef2023.number of first-time 

In [23]:
# merge df on institution and df_s on name using maximum prefix match and if it fails on get_closest fuzzy match
def merge_ipeds_salary_data(iped_data, salary_data):

    def get_closest(name, names_list):
        """Get the closest matching name using fuzzy matching"""
        return max(names_list, key=lambda x: SequenceMatcher(None, name, x).ratio())

    def get_prefix_match(name, names_list):
        """Get the longest prefix match from the list of names"""
        matches = [n for n in names_list if name.startswith(n) or n.startswith(name)]
        if matches:
            return max(matches, key=len)
        return None

    # Create a new column for matched names
    iped_data['matched_name'] = None

    # First try prefix matching
    for idx, row in iped_data.iterrows():
        prefix_match = get_prefix_match(row['institution'], salary_data['name'].tolist())
        if prefix_match:
            iped_data.at[idx, 'matched_name'] = prefix_match
        else:
            # If prefix match fails, use fuzzy matching
            closest_match = get_closest(row['institution'], salary_data['name'].tolist())
            iped_data.at[idx, 'matched_name'] = closest_match

    # After match, get all the salary data for the matched names
    iped_data = iped_data.merge(salary_data, left_on='matched_name', right_on='name', how='left')
    
    # Drop the temporary matching column
    iped_data = iped_data.drop('matched_name', axis=1)

    # Display the first few rows of the merged dataframe
    print("Merged dataframe shape:", iped_data.shape)
    print("\nFirst few rows of merged dataframe:")
    print(iped_data.head())

    return iped_data

In [24]:
salary_data = get_salary_data()

                                              name state       par_mean  \
0  ASA Institute Of Business & Computer Technology    NY   35390.396804   
1                     Abilene Christian University    TX  138760.969806   
2             Abraham Baldwin Agricultural College    GA   80366.661268   
3                        Academy Of Art University    CA  166594.969612   
4                           Adams State University    CO   76121.816340   

   par_median  
0       29000  
1      101000  
2       66000  
3       92300  
4       67200  
(2202, 4)
Index(['name', 'state', 'par_mean', 'par_median'], dtype='object')


In [27]:
final_df = merge_ipeds_salary_data(ipeds_data, salary_data)
len(final_df)
final_df.head()
final_df.to_csv("proxy_data.csv", index=False)

Merged dataframe shape: (40, 23)

First few rows of merged dataframe:
   unitid                              institution  year  pct_pell  \
0  107141                    John Brown University  2023      24.0   
1  110592  California State University-Los Angeles  2023      77.0   
2  110635        University of California-Berkeley  2023      24.0   
3  110644           University of California-Davis  2023      25.0   
4  110653          University of California-Irvine  2023      35.0   

   pct_admitted  admissions_yield  tuition_fees  enrollment_ft  pct_asian  \
0          68.0              37.0       30832.0           1327          2   
1          92.0              12.0        6813.0          17785         10   
2          12.0              45.0       14850.0          31548         31   
3          42.0              17.0       15247.0          30929         28   
4          26.0              22.0       14237.0          28884         35   

   pct_black  ...  enrollment_ft_in_state  pct

In [28]:
def test_proxy_data():
    df = pd.read_csv("proxy_data.csv")

    # print the first 5 rows
    print(df.head())

    # print the shape of the dataframe
    print(df.shape)

    # print the columns of the dataframe
    print(df.columns)

    return df

test_proxy_data()

   unitid                              institution  year  pct_pell  \
0  107141                    John Brown University  2023      24.0   
1  110592  California State University-Los Angeles  2023      77.0   
2  110635        University of California-Berkeley  2023      24.0   
3  110644           University of California-Davis  2023      25.0   
4  110653          University of California-Irvine  2023      35.0   

   pct_admitted  admissions_yield  tuition_fees  enrollment_ft  pct_asian  \
0          68.0              37.0       30832.0           1327          2   
1          92.0              12.0        6813.0          17785         10   
2          12.0              45.0       14850.0          31548         31   
3          42.0              17.0       15247.0          30929         28   
4          26.0              22.0       14237.0          28884         35   

   pct_black  ...  enrollment_ft_in_state  pct_enrollment_ft_in_state  \
0          4  ...                     NaN  

,unitid,institution,year,pct_pell,pct_admitted,admissions_yield,tuition_fees,enrollment_ft,pct_asian,pct_black,...,enrollment_ft_in_state,pct_enrollment_ft_in_state,enrollment_ft_out_of_state,pct_enrollment_ft_out_of_state,enrollment_ft_foreign_countries,pct_enrollment_ft_foreign_countries,name,state,par_mean,par_median
0,107141,John Brown University,2023,24.0,68.0,37.0,30832.0,1327,2,4,...,NaN,NaN,NaN,NaN,NaN,NaN,John Brown University,AR,115449.075521,86800
1,110592,California State University-Los Angeles,2023,77.0,92.0,12.0,6813.0,17785,10,4,...,3581.0,99.0,9.0,0.0,24.0,1.0,"California State University, Los Angeles",CA,52114.623490,36600
2,110635,University of California-Berkeley,2023,24.0,12.0,45.0,14850.0,31548,31,3,...,5271.0,79.0,678.0,10.0,692.0,10.0,"University Of California, Berkeley",CA,194452.191631,114700
3,110644,University of California-Davis,2023,25.0,42.0,17.0,15247.0,30929,28,2,...,5206.0,79.0,348.0,5.0,1020.0,16.0,"University Of California, Davis",CA,158390.081252,109400
4,110653,University of California-Irvine,2023,35.0,26.0,22.0,14237.0,28884,35,2,...,5222.0,77.0,525.0,8.0,1045.0,15.0,"University Of California, Irvine",CA,121501.651414,92100
5,110680,University of California-San Diego,2023,40.0,25.0,22.0,15265.0,32852,31,2,...,5556.0,79.0,668.0,10.0,777.0,11.0,"University Of California, San Diego",CA,176468.657437,111300
6,123651,Vanguard University of Southern California,2023,50.0,65.0,23.0,39950.0,1811,5,5,...,451.0,90.0,49.0,10.0,0.0,0.0,Vanguard University Of Southern California,CA,119919.500769,83200
7,130794,Yale University,2023,20.0,5.0,70.0,64700.0,6805,17,7,...,93.0,6.0,1323.0,81.0,225.0,14.0,Yale University,CT,515437.751971,199700
8,132842,Albizu University-Miami,2023,67.0,100.0,59.0,13128.0,116,1,5,...,NaN,NaN,NaN,NaN,NaN,NaN,University Of Miami,FL,211065.553883,110100
9,134097,Florida State University,2023,21.0,25.0,31.0,5656.0,29011,4,9,...,4908.0,83.0,916.0,16.0,65.0,1.0,Florida State University,FL,139113.519529,100500
